# RNN Tutorial
 

### Contents
1. Introduction to RNN
2. RNN Cell Structure
3. Different types of RNN
4. Sentiment Analysis via RNN
5. Text Generation via LSTM RNN
6. References

## 5. Text Generation via LSTM RNN

In [ ]:
#Importing the dependencies
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils


In [ ]:
#load ascii text and conver to lowercase
filename = "data.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()


In [ ]:
#Extract all the unique characters from the text
chars = sorted(list(set(raw_text)))
char_to_int  = dict((c,i) for i,c in enumerate(chars))

In [ ]:
#Lets Have a look at some data in our character list
chars[:10]

In [ ]:
#Summary of the dataset
n_chars = len(raw_text)
n_vocab = len(chars)
print(f'Total characters: {n_chars}')
print(f'Total Vocab: {n_vocab}')

1. Now is the time to define the training mechanism of the network. 
2. There can be many ways to split the data into chunks of a fixed size which can be fed to the network.
3. The target would be the character next to the chunk end.

4. we will split the book text up into subsequences with a fixed length of 100 characters, an arbitrary length. 
5. We could just as easily split the data up by sentences and pad the shorter sequences and truncate the longer ones.
6. Each training pattern of the network is comprised of 100 time steps of one character (X) followed by one character output (y)
7. When creating these sequences, we slide this window along the whole book one character at a time allowing each character a chance to be learned from the 100 characters that preceded it.

### Prepare the dataset


In [ ]:
#Encode the dataset of input to output pairs as integers
seq_length=100
dataX = []
dataY = []

for i in range(0,n_chars-seq_length,1):
    seq_in = raw_text[i:i+seq_length]
    seq_out = raw_text[i+seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print(f'Total patterns: {n_patterns}')
    

In [ ]:
#Now let's transform the list of input sequences into the form [samples, time steps, features] expected by an LSTM network.
X = numpy.reshape(dataX,(n_patterns,seq_length,1))
#normalize the data to be easily fed to the LSTM
X = X/float(n_vocab)
#One hot encode the output variable
y = np_utils.to_categorical(dataY)


In [ ]:
#Print out the Shape of X
X.shape

In [ ]:
#Print out the Shape of y
y.shape

# Define the LSTM Model using Keras

In [ ]:
#Specify the model 
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1],X.shape[2])))
model.add(Dropout(0.3))
model.add(Dense(y.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam')

In [ ]:
#As the model above takes time to train, we will use model checkpoint feature of keras  to save the training weights periodically 
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]


In [ ]:
#fit the model 
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

## Generating the Text


In [ ]:
#load the saved Weights

filename = "weights-improvement-epoch-loss.hdf5"
model.load_weights(filename)
model.compile(loss="categorical_crossentropy",optimizer='adam')


In [ ]:
#Reverse encode the integers to text
int_to_char = dict((i, c) for i, c in enumerate(chars))


In [ ]:
#Pick a random seed
start = numpy.random.randint(0,len(dataX)-1)
pattern = dataX[start]
print(f"Seed is \n {''.join([int_to_char[value] for value in pattern])}")

In [ ]:
#Generate the text
for i in range(400):
    x  = numpy.reshape(pattern,(1,len(pattern),1))
    x  = x/float(n_vocab)
    prediction = model.predict(x,verbose=0)
    index  = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]